# InitialSetup

In [1]:
import os
import shutil

if not os.path.exists('data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if os.path.exists('OUTPUT/'):
    shutil.rmtree('OUTPUT/')
    
os.makedirs('OUTPUT/')

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, AveragePooling2D, MaxPooling2D, Convolution2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

2023-08-31 12:33:36.811245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Loading dataset

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)
training_data = train_gen.flow_from_directory('data_set/train', target_size=(256,256), 
                                                 batch_size=32, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255)
val_data = val_gen.flow_from_directory('data_set/val', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255)
test_data = test_gen.flow_from_directory('data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

Found 2485 images belonging to 2 classes.
Found 532 images belonging to 2 classes.
Found 535 images belonging to 2 classes.


# Training model from scratch

In [4]:
classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [5]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [6]:
steps_per_epoch = len(training_data)
validation_steps = len(val_data)

classifier.fit(training_data, epochs=45, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=val_data, validation_steps=validation_steps)

Epoch 1/45


2023-08-31 12:33:40.264273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - ETA: 0s - loss: 0.4893 - accuracy: 0.7577

2023-08-31 12:34:36.962838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


78/78 [==============================] - 60s 757ms/step - loss: 0.4893 - accuracy: 0.7577 - val_loss: 0.2389 - val_accuracy: 0.8797
Epoch 2/45
78/78 [==============================] - 58s 741ms/step - loss: 0.2494 - accuracy: 0.9014 - val_loss: 0.1034 - val_accuracy: 0.9680
Epoch 3/45
78/78 [==============================] - 58s 740ms/step - loss: 0.1945 - accuracy: 0.9312 - val_loss: 0.0802 - val_accuracy: 0.9699
Epoch 4/45
78/78 [==============================] - 58s 743ms/step - loss: 0.1795 - accuracy: 0.9360 - val_loss: 0.0864 - val_accuracy: 0.9737
Epoch 5/45
78/78 [==============================] - 58s 738ms/step - loss: 0.1740 - accuracy: 0.9404 - val_loss: 0.0777 - val_accuracy: 0.9774
Epoch 6/45
78/78 [==============================] - 59s 751ms/step - loss: 0.1370 - accuracy: 0.9577 - val_loss: 0.1464 - val_accuracy: 0.9624
Epoch 7/45
78/78 [==============================] - 59s 755ms/step - loss: 0.1552 - accuracy: 0.9461 - val_loss: 0.0711 - val_accuracy: 0.9718
Epoch 8/45

In [7]:
classifier.save("OUTPUT/ScratchModel.h5")

In [8]:
_, accuracy = classifier.evaluate(test_data, steps = validation_steps)

2023-08-31 13:17:05.011605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


17/17 [==============================] - 3s 174ms/step - loss: 0.0524 - accuracy: 0.9794


In [10]:
print('ScratchModel accuracy :',accuracy)

ScratchModel accuracy : 0.9794392585754395
